In [16]:
import numpy as np
import pandas as pd 
import os


In [47]:
speakers = pd.read_csv('speakers_planets.csv')
abstracts = pd.read_csv('abstracts_all.csv')
# housekeeping
abstracts.rename(columns={'1. First Name':'First Name'}, inplace=True)
abstracts.rename(columns={'2. Last Name':'Last Name'}, inplace=True)
abstracts.rename(columns={'9. a) Please enter your TITLE for the 10th Australian Exoplanet Workshop':'Title'}, inplace=True)
abstracts.rename(columns={'9. b) Please enter your ABSTRACT for the 10th Australian Exoplanet Workshop':'Abstract'}, inplace=True)
# use a join to find speakers for thursday 


abstracts.rename(columns={'Last Name':'Thursday'}, inplace=True)
thursday = pd.merge(abstracts, speakers, on='Thursday', how='right')
# get rid of the abstracts that don't have a speaker
thursday = thursday[thursday['Thursday'].notnull()]
thursday.rename(columns={'Thursday':'Last Name'}, inplace=True)

# now do friday
abstracts.rename(columns={'Thursday':'Friday'}, inplace=True)
friday = pd.merge(abstracts, speakers, on='Friday', how='right')
friday = friday[friday['Friday'].notnull()]
friday.rename(columns={'Friday':'Last Name'}, inplace=True)

# now do the same for monday, tuesday, wednesday from speakers_stars.csv
speakers_stars = pd.read_csv('speakers_stars.csv')

abstracts.rename(columns={'Title':'Planets Title'}, inplace=True)
abstracts.rename(columns={'Abstract':'Planets Abstract'}, inplace=True)
abstracts.rename(columns={'8. a) Please enter your TITLE for Stars in Brisbane':'Title'}, inplace=True)
abstracts.rename(columns={'8. b) Please enter your ABSTRACT for Stars in Brisbane':'Abstract'}, inplace=True)



# now do monday, tuesday, wednesday
abstracts.rename(columns={'Friday':'Monday'}, inplace=True)
monday = pd.merge(abstracts, speakers_stars, on='Monday', how='right')
monday = monday[monday['Monday'].notnull()]
monday.rename(columns={'Monday':'Last Name'}, inplace=True)

abstracts.rename(columns={'Monday':'Tuesday'}, inplace=True)
tuesday = pd.merge(abstracts, speakers_stars, on='Tuesday', how='right')
tuesday = tuesday[tuesday['Tuesday'].notnull()]
tuesday.rename(columns={'Tuesday':'Last Name'}, inplace=True)

abstracts.rename(columns={'Tuesday':'Wednesday'}, inplace=True)
wednesday = pd.merge(abstracts, speakers_stars, on='Wednesday', how='right')
wednesday = wednesday[wednesday['Wednesday'].notnull()]
wednesday.rename(columns={'Wednesday':'Last Name'}, inplace=True)




In [48]:
abstracts.keys()

Index(['Email Address', 'First Name', 'Wednesday', '3. Current Institution',
       '4. Career Level', '5. Gender',
       '7. Which workshop are you submitting an abstract for?', 'Title',
       'Planets Title',
       '10. Would you prefer to present a talk or a poster for Stars in Brisbane?',
       '12. Have you registered for the event(s) for which you are submitting the abstract(s) ?',
       '11. If your abstract is not selected for a talk, would you like to present it as a poster?',
       '10. Would you prefer to present a talk or a poster for the 10th Australian Exoplanet Workshop?',
       'Abstract', 'Planets Abstract', 'If your abstract is not'],
      dtype='object')

In [49]:
# both 

def generate_markdown_for_conference(conference_name, days_df, schedule_days,programme="planets"):
    """
    Generate markdown table for a single conference.
    """
    # Initialize the Hugo table content with headers
    hugo_table = f'---\ntitle: Programme - {programme.capitalize()}\n'
    hugo_table += "menu: {main: {weight: 30}}\ndate: 2024-10-08\n---"
    hugo_table += "\n{{% blocks/cover title=" + f'"Programme - {programme.capitalize()}" ' + 'image_anchor="top" height="full" %}}\n'
    hugo_table += f"## {conference_name}\n\n"
    hugo_table += "| Schedule | " + " | ".join([day.capitalize() for day in schedule_days]) + " |\n"
    hugo_table += "| -------- | " + " | ".join(["----------------"] * len(schedule_days)) + " |\n"

    # Combine the schedules from all days
    all_schedules = sorted(set().union(*[set(df['Schedule']) for df in days_df]))

    for schedule in all_schedules:
        row = f"| {schedule} |"
        for df in days_df:
            speaker_row = df[df['Schedule'] == schedule]
            if not speaker_row.empty:
                speaker = speaker_row.iloc[0]
                full_name = f"{speaker['First Name']} {speaker['Last Name']}"
                print(full_name)
                slug = f"{speaker['First Name'].lower().replace(' ','-')}-{speaker['Last Name'].lower().replace(' ', '-')}"
                link = f"[{full_name}](speakers/{slug}/_index.md)"
                row += f" {link} |"
                # Create individual markdown page for the speaker
                create_speaker_markdown(speaker_row,slug,programme=programme)

            else:
                row += " |"  # Empty cell if no speaker at this time
        hugo_table += row + "\n"
    hugo_table += "\n{{% /blocks/cover %}}"

    return hugo_table


def create_speaker_markdown(speaker_row, slug,programme="planets"):
    """Creates a markdown page for an individual speaker."""
    # Extract the actual values from the pandas Series
    first_name = speaker_row['First Name'].values[0] if not speaker_row['First Name'].isnull().all() else ""
    last_name = speaker_row['Last Name'].values[0] if not speaker_row['Last Name'].isnull().all() else ""
    title = speaker_row['Title'].values[0] if not speaker_row['Title'].isnull().all() else "No Title Available"
    if title == "No Title Available":
        try:
            title = speaker_row['Planets Title'].values[0] if not speaker_row['Planets Title'].isnull().all() else "No Title Available"
        except:
            print(f"Warning: No title available for {first_name} {last_name}")
    abstract = speaker_row['Abstract'].values[0] if not speaker_row['Abstract'].isnull().all() else "No Abstract Available"
    if abstract == "No Abstract Available":
        try:
            abstract = speaker_row['Planets Abstract'].values[0] if not speaker_row['Planets Abstract'].isnull().all() else "No Abstract Available"
        except:
            print(f"Warning: No abstract available for {first_name} {last_name}")

    # Full name
    full_name = f"{first_name} {last_name}"

    pre, post = r'{{% blocks/cover title="Placeholder" image_anchor="top" height="full" %}}'.replace("Placeholder", full_name), r"{{% /blocks/cover %}}"

    # Generate the markdown content
    speaker_md = f"""---
title: "{full_name}"
date: 2024-10-08
---

{pre}

## {title}

{abstract}

{post}
                    """
    # Write the speaker's markdown file
    outdir = f"../content/en/programme_{programme}/speakers/"
    os.makedirs(os.path.join(outdir, slug), exist_ok=True)
    speaker_md_filename = os.path.join(outdir, f"{slug}/_index.md".replace(" ", "-"))
    with open(speaker_md_filename, 'w') as speaker_file:
        speaker_file.write(speaker_md)
    # copy featured-background.jpg from here to the speaker's directory using os
    os.system(f"cp featured-background.jpg {os.path.join(outdir, slug)}")
    


def generate_markdown_for_both_conferences(stars_dfs, exoplanets_dfs, stars_days, exoplanets_days):
    # Ensure the speaker directory exists
    os.makedirs("speakers", exist_ok=True)

    # Generate markdown tables for both conferences
    stars_table = generate_markdown_for_conference("Stars in Brisbane", stars_dfs, stars_days,programme="stars")

    # Write the stars markdown table to a file
    with open('../content/en/programme_stars/_index.md', 'w') as file:
        file.write(stars_table)

    exoplanets_table = generate_markdown_for_conference("Australian Exoplanet Workshop", exoplanets_dfs, exoplanets_days,programme="planets")

    # Write the exoplanets markdown table to a file
    with open('../content/en/programme_planets/_index.md', 'w') as file:
        file.write(exoplanets_table)

    print("Markdown schedule and speaker pages generated successfully.")

# Assuming `monday`, `tuesday`, `wednesday`, `thursday`, and `friday` are the dataframes
# Lists of dataframes for each conference
stars_dfs = [monday, tuesday, wednesday]
exoplanets_dfs = [thursday, friday]

# Days for each conference
stars_days = ['monday', 'tuesday', 'wednesday']
exoplanets_days = ['thursday', 'friday']

# Generate the combined markdown file
generate_markdown_for_both_conferences(stars_dfs, exoplanets_dfs, stars_days, exoplanets_days);

Jeremy Bailey
Adithya Balasubramaniam
Laura Venuti
Peter Tuthill
Courtney Crawford
Tania Ahmed
Ana Lourdes Juarez Garcia
Barnali Das
Yuzhe Song
Ryan White
Sarah Martell
Alexander Venner
John Bray
Jacinda Webb
Brad Carter
Alex Wallace
Han Shen
Evgeni Grishin
Daniel Zucker
Emma Brown
Nicolas Rodriguez-Segovia
Richard de Grijs
Ivey Davis
Anuj Gautam
Maksym Mohorian
May Gade Pedersen
Stephen Neilson
Chunliang Mu
Dennis Stello
Erica Thygesen
Jack Nibbs
Laura Driessen
Yuzhe Song
Kayla Martin
Mark Walker
Deepak Chahal
Meghna Menon
Tara Murphy
Taissa Danilovich
Katelyn Smith
Alexander Csukai
Barnali Das
Amanda Karakas
Conaire Deagan
Harmeen Kaur
Shishir Dholakia
Ryosuke Hirai
Sven Buder
Christophe Pinte
Stephen Kane
Evgeni Grishin
Ava Morrissey
Tyler Fairnington
Nicholas Borsato
Sydney Vach
Iain Hammond
Ben Montet
Louis Desdoigts
Sakhee Bhure
Max Charles
Patryk Sofia Lykawka
Brendan McKee
Thomas Hilder
Thomas Plunkett
Peter Tuthill
Rachel Harrison
Jeremy Bailey
Jonti Horner
Jean-Philippe Beaul

In [6]:
# import re
# from pathlib import Path

# def convert_markdown_to_latex(markdown_content):
#     # Convert the header
#     latex_content = r"\documentclass{article}" + "\n" + r"\usepackage{longtable}" + "\n" + r"\usepackage{hyperref}" + "\n" + r"\usepackage{geometry}" + "\n" + r"\geometry{a4paper, margin=1in}" + "\n" + r"\begin{document}" + "\n"
    
#     # Extract title
#     title_match = re.search(r'title:\s*(.*)', markdown_content)
#     if title_match:
#         title = title_match.group(1).strip()
#         latex_content += r"\title{" + title + "}\n" + r"\maketitle" + "\n"
    
#     # Convert Markdown links to LaTeX \href
#     markdown_content = re.sub(r'\[([^\]]+)\]\(([^)]+)\)', r'\\href{\2}{\1}', markdown_content)
    
#     # Extract sections
#     sections = re.split(r'#\s+', markdown_content)
#     for section in sections:
#         if section.strip():
#             # Extract section title
#             section_title_match = re.match(r'([^\n]+)', section)
#             if section_title_match:
#                 section_title = section_title_match.group(1).strip()
#                 latex_content += r"\section{" + section_title + "}\n"
            
#             # Extract tables
#             tables = re.findall(r'\|.*\|', section)
#             for table in tables:
#                 rows = table.strip().split('\n')
#                 latex_content += r"\begin{longtable}{|" + " | ".join(['c'] * len(rows[0].split('|'))) + r"|}" + "\n"
#                 latex_content += r"\hline" + "\n"
#                 for row in rows:
#                     columns = row.split('|')[1:-1]
#                     latex_content += " & ".join([col.strip() for col in columns]) + r" \\" + "\n"
#                     latex_content += r"\hline" + "\n"
#                 latex_content += r"\end{longtable}" + "\n"
    
#     latex_content += r"\end{document}"
#     return latex_content


In [7]:
# from pathlib import Path
# txt = Path('../content/en/programme/_index.md').read_text()

# # Write to a .tex file
# output_path = Path('output.tex')
# output_path.write_text(convert_markdown_to_latex(txt))
